In [5]:
import os
import re
import requests
import uuid
import pymysql
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from datetime import datetime

# 환경변수 로드 (.env에서 DB 설정 읽기)
load_dotenv()

# 로그 출력 함수
def log(message):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{timestamp}] {message}")

# DB에 Club 한 줄 저장
def insert_club(club_name, category, univ_name="금오공과대학교"):
    try:
        conn = pymysql.connect(
            host=os.getenv("DB_HOST"),
            port=int(os.getenv("DB_PORT")),
            user=os.getenv("DB_USER"),
            password=os.getenv("DB_PASSWORD"),
            database=os.getenv("DB_NAME"),
            charset='utf8mb4',
            autocommit=True
        )
        with conn.cursor() as cursor:
            # 중복 방지: 동일 이름 있는지 확인
            cursor.execute("SELECT COUNT(*) FROM Club WHERE clubName = %s", (club_name,))
            if cursor.fetchone()[0] > 0:
                log(f"⚠️ 중복 감지: {club_name} → 건너뜀")
                return

            sql = """
                INSERT INTO Club (clubId, clubName, category, univName, isOfficial)
                VALUES (%s, %s, %s, %s, 1)
            """
            club_id = uuid.uuid4().hex
            cursor.execute(sql, (club_id, club_name, category, univ_name))
        conn.close()
        log(f"✅ Club 저장 완료: {club_name} ({category})")
    except Exception as e:
        log(f"[DB ERROR] Club 저장 실패: {e}")

# 금오공대 동아리 홈페이지 크롤링 → DB 저장
def load_and_insert_clubs():
    url = "https://www.kumoh.ac.kr/ko/sub04_01_02.do"
    headers = {'User-Agent': 'Mozilla/5.0'}
    try:
        response = requests.get(url, headers=headers)
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'html.parser')

        for area in soup.select("div.contents-area"):
            category_tag = area.select_one("h4.title0401")
            table = area.select_one("div.table-type01")
            if not category_tag or not table:
                continue
            category = category_tag.text.strip()
            for row in table.select("tbody tr"):
                cols = row.select("td")
                if not cols:
                    continue
                name = cols[0].text.strip()
                if name:
                    insert_club(name, category)
    except Exception as e:
        log(f"[ERROR] Club 정보 크롤링 실패: {e}")

# 실행
if __name__ == '__main__':
    load_and_insert_clubs()


[2025-05-26 22:44:35] ✅ Club 저장 완료: 금오사진예술연구회 (문화예술 분야)
[2025-05-26 22:44:35] ✅ Club 저장 완료: 불모지대 (문화예술 분야)
[2025-05-26 22:44:35] ✅ Club 저장 완료: 소리샘 (문화예술 분야)
[2025-05-26 22:44:35] ✅ Club 저장 완료: FAZE (문화예술 분야)
[2025-05-26 22:44:38] ✅ Club 저장 완료: 타니말라 (문화예술 분야)
[2025-05-26 22:44:38] ✅ Club 저장 완료: OVERFLOW (문화예술 분야)
[2025-05-26 22:44:38] ✅ Club 저장 완료: C.O.K (문화예술 분야)
[2025-05-26 22:44:38] ✅ Club 저장 완료: 공데쌩 (문화예술 분야)
[2025-05-26 22:44:38] ✅ Club 저장 완료: 밤하늘 (학술 분야)
[2025-05-26 22:44:38] ✅ Club 저장 완료: KITE (학술 분야)
[2025-05-26 22:44:38] ✅ Club 저장 완료: 거북선신화 (학술 분야)
[2025-05-26 22:44:38] ✅ Club 저장 완료: E.C.R.C (학술 분야)
[2025-05-26 22:44:38] ✅ Club 저장 완료: 묵향 (학술 분야)
[2025-05-26 22:44:39] ✅ Club 저장 완료: Auto-Mania (학술 분야)
[2025-05-26 22:44:39] ✅ Club 저장 완료: 치즈 (학술 분야)
[2025-05-26 22:44:39] ✅ Club 저장 완료: 금오테니스 (체육활동 분야)
[2025-05-26 22:44:39] ✅ Club 저장 완료: 태양농구회 (체육활동 분야)
[2025-05-26 22:44:39] ✅ Club 저장 완료: 한알탁구회 (체육활동 분야)
[2025-05-26 22:44:39] ✅ Club 저장 완료: 코브라 (체육활동 분야)
[2025-05-26 22:44:39] ✅ Club 저